# AI-Driven Drug Discovery for SSTR2
### Somatostatin Receptor Type 2 Virtual Screening Pipeline

---

**Project**: PRST_N_FM &nbsp;|&nbsp; **Target**: SSTR2 (신경내분비종양 치료 표적)  
**Tools**: AlphaFold3 + FoldMason + PyMOL + NVIDIA BioNeMo NIM APIs  
**Goal**: 기존 약물(Octreotide)보다 더 강하게 결합하는 새로운 분자 탐색

In [1]:
# ===== Setup (이 셀은 실행만 하면 됩니다) =====
import sys, json, os, warnings
from pathlib import Path
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML, display, Markdown

warnings.filterwarnings('ignore')
if sys.platform == 'darwin':
    mpl.rc('font', family='AppleGothic')
mpl.rc('axes', unicode_minus=False)
plt.style.use('seaborn-v0_8-whitegrid')
mpl.rcParams.update({'figure.dpi': 130, 'figure.figsize': (12, 5)})

REPO = Path('.').resolve().parent
sys.path.insert(0, str(REPO))

# API key check
api_key = os.getenv('NGC_CLI_API_KEY') or os.getenv('NVIDIA_API_KEY')
if not api_key:
    for kf in [REPO/'molmim.key', REPO/'ngc.key']:
        if kf.exists(): api_key = kf.read_text().strip(); break
HAS_API = bool(api_key and api_key.startswith('nvapi-'))

display(HTML('<div style="padding:10px;background:#e8f5e9;border-radius:8px;font-size:14px">' +
    f'Environment ready &nbsp; | &nbsp; API: {"Connected" if HAS_API else "Offline mode"}</div>'))

---
## What is SSTR2?

**SSTR2 (Somatostatin Receptor Type 2)** 는 세포 표면의 G단백질 결합 수용체(GPCR)입니다.

| 항목 | 설명 |
|------|------|
| **기능** | 성장호르몬, 인슐린, 글루카곤 분비 억제 |
| **질환** | 신경내분비종양(NET), 말단비대증 |
| **기존 약물** | Octreotide, Lanreotide, Pasireotide (모두 펩타이드 주사제) |
| **과제** | 경구 투여 가능한 소분자 또는 더 강한 펩타이드 바인더 |

### 우리의 접근법: 3-Arm Parallel Screening

```
               AlphaFold3 (SSTR2 + Somatostatin-14 복합체)
                              |
                    Binding Pocket 분석 (35 잔기)
                    /         |          \
              Arm 1         Arm 2         Arm 3
           소분자 설계    펩타이드 변이    De Novo 설계
              |              |              |
           MolMIM      Ala-scanning    RFdiffusion
              |              |              |
           DiffDock     FlexPepDock    ProteinMPNN
              |              |              |
              +------+------+          ESMFold
                     |                     |
                통합 랭킹 & 후보 선정
```

---
## 1. AlphaFold3: 단백질 복합체 구조 예측

Google DeepMind의 **AlphaFold3**로 SSTR2와 Somatostatin-14의 결합 구조를 예측했습니다.  
5개 모델 중 **Model 0**이 가장 높은 신뢰도(Ranking Score = 0.83)를 보였습니다.

In [ ]:
# 3D 단백질 구조 시각화
try:
    import py3Dmol
except ImportError:
    print("⚠ py3Dmol 미설치. 설치: pip install py3Dmol")
    py3Dmol = None

if py3Dmol:
    pdb_path = REPO / 'data' / 'fold_test1' / 'fold_test1_model_0.pdb'
    pdb_data = pdb_path.read_text()

    view = py3Dmol.view(width=800, height=500)
    view.addModel(pdb_data, 'pdb')

    # Chain B (SSTR2) - 수용체를 연한 파란색 cartoon
    view.setStyle({'chain': 'B'}, {'cartoon': {'color': '#6495ED', 'opacity': 0.85}})
    # Chain A (Somatostatin) - 리간드를 주황색 두꺼운 cartoon + sticks
    view.setStyle({'chain': 'A'}, {
        'cartoon': {'color': '#FF6347', 'thickness': 0.4},
        'stick': {'color': '#FF6347', 'radius': 0.15}
    })

    # 표면 (수용체)
    view.addSurface(py3Dmol.VDW, {'opacity': 0.15, 'color': '#B0C4DE'}, {'chain': 'B'})

    view.zoomTo()
    view.setBackgroundColor('#FFFFFF')

    display(HTML('<h4 style="text-align:center;color:#333">SSTR2 (파란색) + Somatostatin-14 (주황색) 복합체</h4>'))
    view.show()

ModuleNotFoundError: No module named 'py3Dmol'

In [ ]:
# AlphaFold3 신뢰도 지표
conf_dir = REPO / 'data' / 'fold_test1'
confidences = [json.loads((conf_dir / f'fold_test1_summary_confidences_{i}.json').read_text()) for i in range(5)]

fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

scores = [c['ranking_score'] for c in confidences]
colors_rank = ['#FF6347' if i == np.argmax(scores) else '#B0C4DE' for i in range(5)]

# Ranking Score
ax = axes[0]
bars = ax.bar(range(5), scores, color=colors_rank, edgecolor='white', linewidth=1.5, width=0.6)
for b, s in zip(bars, scores): ax.text(b.get_x()+b.get_width()/2, s+0.02, f'{s:.2f}', ha='center', fontsize=11, fontweight='bold')
ax.set_xticks(range(5)); ax.set_xticklabels([f'Model {i}' for i in range(5)])
ax.set_ylabel('Score'); ax.set_ylim(0, 1)
ax.set_title('Ranking Score (높을수록 좋음)', fontweight='bold', fontsize=12)

# pTM vs ipTM
ax = axes[1]; x = np.arange(5); w = 0.35
ax.bar(x-w/2, [c['ptm'] for c in confidences], w, label='pTM (단일체)', color='#6495ED', edgecolor='white')
ax.bar(x+w/2, [c['iptm'] for c in confidences], w, label='ipTM (계면)', color='#FF6347', edgecolor='white')
ax.set_xticks(x); ax.set_xticklabels([f'M{i}' for i in range(5)])
ax.set_ylabel('Score'); ax.set_ylim(0, 1); ax.legend(fontsize=9)
ax.set_title('구조 신뢰도 (pTM) vs 결합 신뢰도 (ipTM)', fontweight='bold', fontsize=12)

# PAE heatmap
ax = axes[2]
pae = np.array(confidences[0]['chain_pair_pae_min'])
im = ax.imshow(pae, cmap='RdYlGn_r', vmin=0, vmax=6)
ax.set_xticks([0,1]); ax.set_yticks([0,1])
ax.set_xticklabels(['Somatostatin', 'SSTR2'], fontsize=10)
ax.set_yticklabels(['Somatostatin', 'SSTR2'], fontsize=10)
for (j,i), v in np.ndenumerate(pae): ax.text(i, j, f'{v:.1f}Å', ha='center', va='center', fontsize=13, fontweight='bold')
plt.colorbar(im, ax=ax, label='PAE (Å) — 낮을수록 정확', shrink=0.8)
ax.set_title('결합 정확도 (PAE)', fontweight='bold', fontsize=12)

plt.suptitle('AlphaFold3 예측 결과: 5개 모델 비교', fontweight='bold', fontsize=14, y=1.03)
plt.tight_layout(); plt.show()

display(Markdown(f"""
> **Best Model (Model 0)**: Ranking={scores[0]:.2f}, ipTM={confidences[0]['iptm']:.2f}  
> ipTM > 0.7 → 단백질 간 결합 구조가 신뢰할 만함
"""))

---
## 2. FoldMason: 구조 일관성 검증

**FoldMason**은 여러 단백질 구조를 3D 좌표 기반으로 정렬하고, 모델 간 얼마나 일관성 있는지(lDDT) 평가합니다.  
AlphaFold3의 5개 모델 중 3개(Model 0, 1, 2)를 비교했습니다.

| 측정 항목 | 설명 |
|-----------|------|
| **lDDT** (Local Distance Difference Test) | 0~1 사이 값. 1에 가까울수록 구조가 일치 |
| **3Di Alphabet** | 3D 구조를 문자로 변환한 "구조 서열" (Foldseek 기반) |
| **Guide Tree** | 구조 유사도 기반 계통수 |

In [ ]:
# FoldMason 구조 정렬 결과
msa_aa = (REPO / 'results' / 'foldmason' / 'result_foldmason_aa.fa').read_text()
msa_3di = (REPO / 'results' / 'foldmason' / 'result_foldmason_3di.fa').read_text()
newick = (REPO / 'results' / 'foldmason' / 'result_foldmason.nw').read_text().strip()

# FASTA 파싱
def parse_fasta(text):
    seqs = {}
    name = None
    for line in text.strip().split('\n'):
        if line.startswith('>'):
            name = line[1:].strip()
            seqs[name] = ''
        elif name:
            seqs[name] += line.strip()
    return seqs

aa_seqs = parse_fasta(msa_aa)
di_seqs = parse_fasta(msa_3di)

display(HTML(f'''
<div style="background:#f3f4f6;padding:20px;border-radius:12px;margin:10px 0">
  <h3 style="margin-top:0">FoldMason 구조 정렬 결과</h3>
  <table style="font-size:14px;border-collapse:collapse">
    <tr><td style="padding:6px 16px;font-weight:bold">Average MSA lDDT</td>
        <td style="padding:6px 16px"><span style="font-size:24px;color:#2e7d32;font-weight:bold">0.664</span>
        <span style="color:#888"> (0.5~0.7: 중간 수준 일관성)</span></td></tr>
    <tr><td style="padding:6px 16px;font-weight:bold">정렬된 서열</td>
        <td style="padding:6px 16px">{len(aa_seqs)}개 (3 모델 x 2 체인)</td></tr>
    <tr><td style="padding:6px 16px;font-weight:bold">Guide Tree</td>
        <td style="padding:6px 16px;font-family:monospace;font-size:11px">{newick}</td></tr>
  </table>
</div>
'''))

In [ ]:
# SSTR2 (Chain B) 서열 보존도 분석
chain_b_seqs = {k: v for k, v in aa_seqs.items() if '_B' in k}
chain_b_names = list(chain_b_seqs.keys())
chain_b_aligned = list(chain_b_seqs.values())
seq_len = len(chain_b_aligned[0])

# 잔기별 보존도 (3개 모델 간 동일 잔기 비율)
conservation = []
for pos in range(seq_len):
    residues = [s[pos] for s in chain_b_aligned if pos < len(s)]
    if all(r == '-' for r in residues):
        conservation.append(np.nan)
    else:
        most_common = Counter(residues).most_common(1)[0][1]
        conservation.append(most_common / len(residues))

conservation = np.array(conservation, dtype=float)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 6), gridspec_kw={'height_ratios': [2, 1]})

# 보존도 프로파일
valid = ~np.isnan(conservation)
positions = np.arange(seq_len)[valid]
cons_valid = conservation[valid]
colors_cons = ['#2e7d32' if c == 1.0 else '#ff9800' if c >= 0.66 else '#e53935' for c in cons_valid]
ax1.bar(positions, cons_valid, color=colors_cons, width=1.0, edgecolor='none')
ax1.set_ylabel('보존도', fontsize=12)
ax1.set_title('SSTR2 구조 정렬: 잔기별 보존도 (3 모델 비교)', fontweight='bold', fontsize=13)
ax1.set_ylim(0, 1.1)
ax1.axhline(y=1.0, color='#2e7d32', linestyle='--', alpha=0.3)

# 범례
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2e7d32', label='완전 보존 (100%)'),
    Patch(facecolor='#ff9800', label='부분 보존 (67%)'),
    Patch(facecolor='#e53935', label='가변 (33%)')
]
ax1.legend(handles=legend_elements, fontsize=9, loc='lower right')

# 3Di 알파벳 비교
chain_b_3di = {k: v for k, v in di_seqs.items() if '_B' in k}
di_aligned = list(chain_b_3di.values())

# 3Di 보존도
di_cons = []
for pos in range(min(len(s) for s in di_aligned)):
    residues = [s[pos] for s in di_aligned]
    if all(r == '-' for r in residues):
        di_cons.append(np.nan)
    else:
        most_common = Counter(residues).most_common(1)[0][1]
        di_cons.append(most_common / len(residues))
di_cons = np.array(di_cons, dtype=float)
di_valid = ~np.isnan(di_cons)
di_positions = np.arange(len(di_cons))[di_valid]
di_colors = ['#1565c0' if c == 1.0 else '#90caf9' for c in di_cons[di_valid]]
ax2.bar(di_positions, di_cons[di_valid], color=di_colors, width=1.0, edgecolor='none')
ax2.set_ylabel('3Di 보존도', fontsize=12)
ax2.set_xlabel('정렬 위치', fontsize=12)
ax2.set_ylim(0, 1.1)
ax2.set_title('3Di 구조 알파벳 보존도 (구조적 특징의 일관성)', fontweight='bold', fontsize=11)

plt.tight_layout()
plt.show()

n_conserved = np.sum(conservation[valid] == 1.0)
n_total = int(np.sum(valid))
display(Markdown(f"""
> **해석**: SSTR2 서열의 **{n_conserved}/{n_total}** 위치가 3개 모델에서 완전히 보존됨  
> 3Di 알파벳 보존 = 3D 구조적 환경도 모델 간 일치한다는 의미  
> **→ 바인딩 포켓 영역의 구조적 신뢰도가 높아, 도킹 분석에 적합**
"""))

In [ ]:
# 3개 모델 중첩 3D 시각화 (PyMOL 대체)
if py3Dmol:
    view2 = py3Dmol.view(width=800, height=450)

    model_colors = ['#2e7d32', '#1565c0', '#e91e63']  # green, blue, pink
    model_names = ['Model 0', 'Model 1', 'Model 2']

    for i, (color, name) in enumerate(zip(model_colors, model_names)):
        pdb = (REPO / 'data' / 'fold_test1' / f'fold_test1_model_{i}.pdb').read_text()
        view2.addModel(pdb, 'pdb')
        view2.setStyle({'model': i, 'chain': 'B'}, {'cartoon': {'color': color, 'opacity': 0.7}})
        view2.setStyle({'model': i, 'chain': 'A'}, {'cartoon': {'color': color, 'opacity': 0.4, 'thickness': 0.2}})

    view2.zoomTo()
    view2.setBackgroundColor('#FFFFFF')

    display(HTML('''
    <h4 style="text-align:center;color:#333">3개 AlphaFold3 모델 중첩 (FoldMason 정렬 검증)</h4>
    <p style="text-align:center;font-size:13px;color:#666">
        <span style="color:#2e7d32">■</span> Model 0 &nbsp;&nbsp;
        <span style="color:#1565c0">■</span> Model 1 &nbsp;&nbsp;
        <span style="color:#e91e63">■</span> Model 2
    </p>
    '''))
    view2.show()
else:
    print("⚠ py3Dmol 미설치 — 3D 뷰 건너뜀")

---
## 3. Binding Pocket: 약물이 결합하는 자리

Somatostatin-14(리간드)이 SSTR2(수용체)에 결합하는 **바인딩 포켓**을 분석했습니다.  
리간드에서 **5Å 이내**에 있는 수용체 잔기 **35개**를 추출했습니다.

In [ ]:
# 바인딩 포켓 3D 시각화
pocket = json.loads((REPO / 'results' / 'sstr2_docking' / 'binding_pocket.json').read_text())

if not py3Dmol:
    print("⚠ py3Dmol 미설치 — 3D 뷰 건너뜀")
else:
    view3 = py3Dmol.view(width=800, height=500)
    view3.addModel(pdb_data, 'pdb')

    # 수용체 회색 투명
    view3.setStyle({'chain': 'B'}, {'cartoon': {'color': '#D3D3D3', 'opacity': 0.3}})
    # 리간드 주황색
    view3.setStyle({'chain': 'A'}, {
        'cartoon': {'color': '#FF6347', 'thickness': 0.4},
        'stick': {'color': '#FF6347', 'radius': 0.15}
    })

    # 포켓 잔기 하이라이팅
    for r in pocket['pocket_residues']:
        view3.setStyle({'chain': r['chain'], 'resi': r['resid']},
                       {'cartoon': {'color': '#4169E1', 'opacity': 0.9},
                        'stick': {'color': '#4169E1', 'radius': 0.12}})

    # 핵심 잔기 라벨
    key_residues = [('B', 122, 'ASP'), ('B', 184, 'ARG'), ('B', 197, 'TRP'),
                    ('B', 205, 'TYR'), ('B', 272, 'PHE'), ('B', 294, 'PHE')]
    for ch, resid, resname in key_residues:
        view3.addLabel(f'{resname}{resid}', {
            'fontSize': 10, 'backgroundColor': '#333333', 'fontColor': '#FFFFFF',
            'backgroundOpacity': 0.7
        }, {'chain': ch, 'resi': resid})

    view3.zoomTo({'chain': 'A'})
    view3.setBackgroundColor('#FFFFFF')

    display(HTML('''
    <h4 style="text-align:center;color:#333">SSTR2 Binding Pocket (파란색: 포켓 잔기 35개, 주황색: Somatostatin)</h4>
    <p style="text-align:center;font-size:12px;color:#888">마우스로 회전/확대 가능</p>
    '''))
    view3.show()

In [ ]:
# 핵심 상호작용 & 포켓 구성
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16, 5))

# 1. 포켓 잔기 유형 분포
resnames = [r['resname'] for r in pocket['pocket_residues']]
aa_props = {
    'hydrophobic': ['ALA','VAL','ILE','LEU','MET','PHE','TRP','PRO'],
    'polar': ['SER','THR','ASN','GLN','TYR','CYS'],
    'positive': ['LYS','ARG','HIS'],
    'negative': ['ASP','GLU'],
    'special': ['GLY'],
}
prop_colors_map = {'hydrophobic':'#E07B54','polar':'#4C9A2A','positive':'#4169E1','negative':'#DC143C','special':'#888'}
def get_prop(rn):
    for p, aas in aa_props.items():
        if rn in aas: return p
    return 'special'

prop_counts = Counter(get_prop(r) for r in resnames)
labels_p = list(prop_counts.keys())
sizes_p = list(prop_counts.values())
colors_p = [prop_colors_map[l] for l in labels_p]
label_kr = {'hydrophobic':'소수성','polar':'극성','positive':'양전하','negative':'음전하','special':'기타'}
ax1.pie(sizes_p, labels=[f"{label_kr.get(l,l)}\n({s}개)" for l,s in zip(labels_p, sizes_p)],
        colors=colors_p, autopct='%1.0f%%', startangle=90,
        textprops={'fontsize': 11}, wedgeprops={'edgecolor': 'white', 'linewidth': 2})
ax1.set_title('포켓 잔기 화학적 성질', fontweight='bold', fontsize=12)

# 2. Somatostatin 핵심 상호작용
interactions = [
    ('Trp8\n(가장 깊은 삽입)', 13, '#e53935'),
    ('Lys9\n(가장 가까운 접촉)', 9, '#1565c0'),
    ('Phe7', 8, '#ff9800'),
    ('Thr10', 5, '#4caf50'),
    ('Phe6', 4, '#9c27b0'),
    ('Thr12', 3, '#607d8b'),
    ('Ala1', 1, '#bdbdbd'),
]
names_i, vals_i, colors_i = zip(*interactions)
bars = ax2.barh(range(len(names_i)), vals_i, color=colors_i, edgecolor='white', linewidth=1.5, height=0.6)
ax2.set_yticks(range(len(names_i))); ax2.set_yticklabels(names_i, fontsize=10)
ax2.invert_yaxis()
ax2.set_xlabel('접촉 SSTR2 잔기 수', fontsize=11)
ax2.set_title('Somatostatin 잔기별 접촉 수', fontweight='bold', fontsize=12)
for i, v in enumerate(vals_i): ax2.text(v+0.2, i, str(v), va='center', fontsize=10, fontweight='bold')

# 3. 접촉 거리
distances = [
    ('Lys9', 2.58), ('Thr12', 2.87), ('Trp8', 2.91),
    ('Phe7', 3.11), ('Phe6', 3.27), ('Ala1', 3.29), ('Thr10', 3.42)
]
names_d, vals_d = zip(*distances)
colors_d = ['#e53935' if v < 3.0 else '#ff9800' if v < 3.3 else '#4caf50' for v in vals_d]
ax3.barh(range(len(names_d)), vals_d, color=colors_d, edgecolor='white', linewidth=1.5, height=0.6)
ax3.set_yticks(range(len(names_d))); ax3.set_yticklabels(names_d, fontsize=10)
ax3.invert_yaxis()
ax3.set_xlabel('최소 접촉 거리 (Å)', fontsize=11)
ax3.set_title('Somatostatin ↔ SSTR2 최소 거리', fontweight='bold', fontsize=12)
ax3.axvline(x=3.0, color='#e53935', linestyle='--', alpha=0.5, label='강한 상호작용 (< 3Å)')
ax3.legend(fontsize=8)
for i, v in enumerate(vals_d): ax3.text(v+0.03, i, f'{v:.2f}Å', va='center', fontsize=9)

plt.suptitle('SSTR2 바인딩 포켓 상세 분석', fontweight='bold', fontsize=14, y=1.02)
plt.tight_layout(); plt.show()

display(Markdown("""
> **핵심 발견**: **Trp8** (13개 잔기와 접촉, 가장 깊이 삽입)과 **Lys9** (최소 2.58Å, 가장 가까운 접촉)이  
> Somatostatin-SSTR2 결합의 핵심 잔기. 새 약물은 이 상호작용을 보존해야 함.
"""))

---
## 4. Arm 1: AI 소분자 약물 설계

**MolMIM** (NVIDIA)으로 기존 SSTR2 약물에서 새로운 소분자를 생성하고,  
**DiffDock**으로 SSTR2에 자동 도킹합니다.

### 시드 분자 (PubChem 검증)
| 약물 | 특징 | PubChem CID |
|------|------|-------------|
| **Paltusotine** | 최초 FDA 승인 경구 SSTR2 작용제 | 134168328 |
| **L-054522** | Merck 비펩타이드 SSTR2 작용제 | 15965425 |
| **Pasireotide** | 다중 SST 수용체 작용제 (주사제) | 9941444 |
| **Octreotide** | 환형 소마토스타틴 유사체 (주사제) | 448601 |

In [ ]:
# MolMIM 다단계 최적화 결과
opt_file = REPO / 'bionemo' / 'result_optimization_20260209_170244.json'
opt = json.loads(opt_file.read_text())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 최적화 진행
rounds = [r['round'] for r in opt['round_results']]
best_scores = [r['best']['score'] for r in opt['round_results']]
best_smiles = [r['best']['sample'] for r in opt['round_results']]

ax1.plot(rounds, best_scores, 'o-', color='#1565c0', linewidth=3, markersize=14, markerfacecolor='white',
         markeredgecolor='#1565c0', markeredgewidth=3, zorder=5)
ax1.fill_between(rounds, 0, best_scores, alpha=0.1, color='#1565c0')
ax1.axhline(y=0.9, color='#2e7d32', linestyle='--', linewidth=2, alpha=0.6, label='Drug-like (QED > 0.9)')
for r, s in zip(rounds, best_scores):
    ax1.annotate(f'{s:.3f}', (r, s), textcoords='offset points',
                 xytext=(0, 18), ha='center', fontsize=13, fontweight='bold', color='#1565c0')
ax1.set_xlabel('최적화 라운드', fontsize=12)
ax1.set_ylabel('QED Score (약물 유사도)', fontsize=12)
ax1.set_title('MolMIM CMA-ES 다단계 최적화', fontweight='bold', fontsize=13)
ax1.set_ylim(0, 1); ax1.set_xticks(rounds)
ax1.legend(fontsize=10)

# 파이프라인 설명
ax2.axis('off')
process = [
    ('시드 분자', opt['seed'], '#e3f2fd'),
    ('Round 1 최고', best_smiles[0][:35]+'...', '#fff3e0'),
    ('Round 2 최고\n(최종 후보)', best_smiles[1][:35]+'...', '#e8f5e9'),
]
for i, (title, smi, bg) in enumerate(process):
    y = 0.8 - i * 0.3
    ax2.add_patch(plt.Rectangle((0.05, y-0.08), 0.9, 0.18, facecolor=bg,
                                edgecolor='#bdbdbd', linewidth=1.5, transform=ax2.transAxes, zorder=2))
    ax2.text(0.5, y+0.03, title, transform=ax2.transAxes,
             ha='center', fontsize=11, fontweight='bold', zorder=3)
    ax2.text(0.5, y-0.04, smi, transform=ax2.transAxes,
             ha='center', fontsize=9, fontfamily='monospace', color='#555', zorder=3)
    if i < len(process) - 1:
        ax2.annotate('', xy=(0.5, y-0.1), xytext=(0.5, y-0.14),
                     transform=ax2.transAxes,
                     arrowprops=dict(arrowstyle='->', color='#1565c0', lw=2))

ax2.set_title('최적화 경로', fontweight='bold', fontsize=13)

plt.tight_layout(); plt.show()

display(Markdown(f"""
> **결과**: Coumarin(QED~0.5) → 2라운드 최적화 → **QED = {opt['best_overall']['score']:.3f}**  
> Drug-like 기준(0.9) 초과하는 약물 후보 생성 성공
"""))

---
## 5. Arm 2: 펩타이드 공학 — Somatostatin 변이체

Somatostatin-14의 핵심 약효단(Phe7-Trp8-Lys9-Thr10)을 분석하기 위해  
**Alanine scanning** (각 잔기를 Alanine으로 치환)과 **강화 변이체**를 설계했습니다.

In [ ]:
# 변이체 분석
arm2 = json.loads((REPO / 'results' / 'sstr2_docking' / 'arm2_flexpep' / 'arm2_results_20260209_235308.json').read_text())
WILDTYPE = arm2['wildtype']

fig, ax = plt.subplots(figsize=(15, 8))

variants = arm2['results']
n_var = len(variants)
n_res = len(WILDTYPE)

# 잔기별 색상 매트릭스
pharmacophore_pos = {5, 6, 7, 8, 9, 10}  # 0-indexed
disulfide_pos = {2, 13}  # Cys3, Cys14

for i, v in enumerate(variants):
    seq = v['sequence']
    for j in range(min(len(seq), n_res)):
        # 배경색 결정
        if j < len(WILDTYPE) and j < len(seq) and seq[j] != WILDTYPE[j]:
            bg = '#e53935'; fg = 'white'; weight = 'bold'
        elif j in pharmacophore_pos:
            bg = '#bbdefb'; fg = '#1565c0'; weight = 'bold'
        elif j in disulfide_pos:
            bg = '#fff9c4'; fg = '#f57f17'; weight = 'bold'
        else:
            bg = '#f5f5f5'; fg = '#333'; weight = 'normal'

        ax.add_patch(plt.Rectangle((j, n_var-1-i), 1, 1, facecolor=bg, edgecolor='white', linewidth=1))
        ax.text(j+0.5, n_var-1-i+0.5, seq[j], ha='center', va='center',
                fontsize=11, fontweight=weight, color=fg, fontfamily='monospace')

# 축 설정
ax.set_xlim(0, n_res); ax.set_ylim(0, n_var)
ax.set_xticks([j+0.5 for j in range(n_res)])
ax.set_xticklabels([f'{WILDTYPE[j]}\n{j+1}' for j in range(n_res)], fontsize=10)
ax.set_yticks([n_var-1-i+0.5 for i in range(n_var)])

# 이름에 이모지 추가
ylabels = []
for v in variants:
    if v['name'] == 'wildtype': ylabels.append('wildtype (기준)')
    elif 'enhanced' in v['name']: ylabels.append(f"{v['name']} (강화)")
    elif 'octreotide' in v['name']: ylabels.append(f"{v['name']} (기존약)")
    else: ylabels.append(f"{v['name']} (Ala scan)")
ax.set_yticklabels(ylabels, fontsize=10)
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')
ax.set_xlabel('Somatostatin-14 서열 위치', fontsize=12, fontweight='bold')

# 범례
legend_elements = [
    Patch(facecolor='#e53935', edgecolor='white', label='돌연변이 위치'),
    Patch(facecolor='#bbdefb', edgecolor='white', label='약효단 (Pharmacophore)'),
    Patch(facecolor='#fff9c4', edgecolor='white', label='이황화결합 (Cys-Cys)'),
    Patch(facecolor='#f5f5f5', edgecolor='#ddd', label='비변이 위치'),
]
ax.legend(handles=legend_elements, loc='lower center', ncol=4, fontsize=10,
          bbox_to_anchor=(0.5, -0.08), frameon=True)

ax.set_title('Somatostatin-14 변이체 라이브러리 (13개)', fontweight='bold', fontsize=14, pad=15)
plt.tight_layout()
plt.show()

display(Markdown("""
> **Alanine Scanning**: 핵심 잔기(F7, W8, K9)를 Alanine으로 치환 → 결합력 감소 예상 → 해당 잔기의 중요도 검증  
> **강화 변이체**: F→Y (수소결합 추가), K→R (양전하 강화) → 결합력 향상 기대
"""))

---
## 6. Arm 3: AI De Novo 펩타이드 바인더 설계

기존 리간드와 **완전히 무관한** 새로운 펩타이드를 AI로 설계하는 3단계 파이프라인입니다.

| 단계 | 도구 | 역할 |
|------|------|------|
| Step 1 | **RFdiffusion** (Baker Lab) | SSTR2 표면에 맞는 펩타이드 백본(뼈대) 생성 |
| Step 2 | **ProteinMPNN** (Baker Lab) | 백본에 최적 아미노산 서열 배정 |
| Step 3 | **ESMFold** (Meta) | 설계 서열이 의도대로 접히는지 검증 |

In [ ]:
# Arm 3 결과
arm3 = json.loads((REPO / 'results' / 'sstr2_docking' / 'arm3_denovo' / 'arm3_final_20260210_000106.json').read_text())
designs = arm3['designs']

# 파이프라인 퍼널
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. 파이프라인 퍼널 차트
stages = ['RFdiffusion\n(백본 설계)', 'ProteinMPNN\n(서열 설계)', 'ESMFold\n(구조 검증)', 'pLDDT ≥ 70\n(고신뢰 후보)']
counts = [arm3['total_backbones'], arm3['total_designed'], arm3['verified'],
          sum(1 for d in designs if d['plddt'] >= 70)]
stage_colors = ['#e3f2fd', '#bbdefb', '#90caf9', '#42a5f5']

# 가로 퍼널
for i, (stage, count, color) in enumerate(zip(stages, counts, stage_colors)):
    width = count / max(counts) * 0.8 + 0.2
    y_center = len(stages) - 1 - i
    ax1.barh(y_center, width, height=0.7, color=color, edgecolor='white', linewidth=2)
    ax1.text(width + 0.02, y_center, f'{count}개', va='center', fontsize=14, fontweight='bold', color='#1565c0')

ax1.set_yticks(range(len(stages)))
ax1.set_yticklabels(reversed(stages), fontsize=11)
ax1.set_xlim(0, 1.3); ax1.set_xlabel('')
ax1.set_title('파이프라인 진행 현황', fontweight='bold', fontsize=13)
ax1.xaxis.set_visible(False)

# 2. pLDDT 분포 + 개별 포인트
plddts = [d['plddt'] for d in designs]
bb_ids = [d['backbone_idx'] for d in designs]
bb_colors_map = {0: '#e53935', 1: '#1565c0', 2: '#2e7d32', 3: '#ff9800'}
scatter_colors = [bb_colors_map[b] for b in bb_ids]

ax2.scatter(range(len(plddts)), sorted(plddts, reverse=True), c=[bb_colors_map[bb_ids[i]] for i in np.argsort(plddts)[::-1]],
            s=120, edgecolor='white', linewidth=1.5, zorder=5)
ax2.axhline(y=70, color='#2e7d32', linestyle='--', linewidth=2, alpha=0.7, label='고신뢰 기준 (pLDDT=70)')
ax2.axhline(y=50, color='#ff9800', linestyle='--', linewidth=1.5, alpha=0.5, label='최소 기준 (pLDDT=50)')
ax2.fill_between(range(len(plddts)), 70, 100, alpha=0.08, color='#2e7d32')
ax2.set_xlabel('설계 순위 (pLDDT 내림차순)', fontsize=11)
ax2.set_ylabel('pLDDT (구조 신뢰도)', fontsize=11)
ax2.set_title('ESMFold 폴딩 검증 결과', fontweight='bold', fontsize=13)
ax2.set_ylim(40, 90); ax2.legend(fontsize=9)

# 범례
for bb, color in bb_colors_map.items():
    ax2.scatter([], [], c=color, s=80, label=f'Backbone {bb}', edgecolor='white')
ax2.legend(fontsize=8, loc='lower left', ncol=2)

plt.suptitle('Arm 3: De Novo 펩타이드 바인더 설계 결과', fontweight='bold', fontsize=14, y=1.02)
plt.tight_layout(); plt.show()

In [ ]:
# Top 설계 서열 테이블
sorted_designs = sorted(designs, key=lambda x: x['plddt'], reverse=True)

html_rows = ''
for i, d in enumerate(sorted_designs[:8]):
    plddt = d['plddt']
    bg = '#e8f5e9' if plddt >= 70 else '#fff3e0'
    badge = '<span style="background:#2e7d32;color:white;padding:2px 8px;border-radius:10px;font-size:11px">HIGH</span>' if plddt >= 70 else ''
    html_rows += f'''
    <tr style="background:{bg}">
        <td style="padding:8px;text-align:center;font-weight:bold">{i+1}</td>
        <td style="padding:8px;font-family:monospace;font-size:13px">{d['binder_sequence']}</td>
        <td style="padding:8px;text-align:center">{len(d['binder_sequence'])}</td>
        <td style="padding:8px;text-align:center;font-weight:bold">{plddt:.1f}</td>
        <td style="padding:8px;text-align:center">Backbone {d['backbone_idx']}</td>
        <td style="padding:8px;text-align:center">{badge}</td>
    </tr>'''

display(HTML(f'''
<h4>Top 8 De Novo 펩타이드 바인더 후보</h4>
<table style="border-collapse:collapse;width:100%;font-size:13px;border:1px solid #ddd">
    <thead>
        <tr style="background:#1565c0;color:white">
            <th style="padding:10px">순위</th>
            <th style="padding:10px">서열</th>
            <th style="padding:10px">길이</th>
            <th style="padding:10px">pLDDT</th>
            <th style="padding:10px">백본</th>
            <th style="padding:10px">신뢰도</th>
        </tr>
    </thead>
    <tbody>{html_rows}</tbody>
</table>
<p style="font-size:12px;color:#888;margin-top:8px">pLDDT > 70: 설계 서열이 의도한 3D 구조로 접힐 가능성이 높음</p>
'''))

In [ ]:
# Top 설계 펩타이드 3D 구조 (ESMFold 결과)
best_design = sorted_designs[0]
best_pdb_path = REPO / 'results' / 'sstr2_docking' / 'arm3_denovo' / f"esmfold_bb{best_design['backbone_idx']:02d}_seq{best_design['seq_idx']}.pdb"

if not py3Dmol:
    print("⚠ py3Dmol 미설치 — 3D 뷰 건너뜀")
elif best_pdb_path.exists():
    best_pdb = best_pdb_path.read_text()
    view4 = py3Dmol.view(width=800, height=400)
    view4.addModel(best_pdb, 'pdb')
    view4.setStyle({'model': 0}, {'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'rwb', 'min': 40, 'max': 90}}})
    view4.addSurface(py3Dmol.VDW, {'opacity': 0.15, 'colorscheme': {'prop': 'b', 'gradient': 'rwb', 'min': 40, 'max': 90}})
    view4.zoomTo()
    view4.setBackgroundColor('#FFFFFF')

    display(HTML(f'''
    <h4 style="text-align:center;color:#333">Best Design: {best_design['binder_sequence']} (pLDDT={best_design['plddt']:.1f})</h4>
    <p style="text-align:center;font-size:12px;color:#888">색상: pLDDT 기반 (파란색=높은 신뢰도, 빨간색=낮은 신뢰도)</p>
    '''))
    view4.show()
else:
    print(f'PDB 파일 없음: {best_pdb_path}')

---
## 7. Summary & Next Steps

In [ ]:
# 최종 대시보드
fig = plt.figure(figsize=(16, 8))
gs = fig.add_gridspec(2, 3, hspace=0.35, wspace=0.3)

# 1. 파이프라인 개요
ax = fig.add_subplot(gs[0, 0])
ax.axis('off')
summary_items = [
    ('AlphaFold3', f'Ranking = {scores[0]:.2f}', '#e3f2fd'),
    ('FoldMason', 'lDDT = 0.664', '#fce4ec'),
    ('Pocket', f'{pocket["num_pocket_residues"]} residues', '#e8f5e9'),
]
for i, (name, val, bg) in enumerate(summary_items):
    y = 0.85 - i * 0.32
    ax.add_patch(plt.Rectangle((0.05, y-0.08), 0.9, 0.22, facecolor=bg,
                                edgecolor='#bdbdbd', linewidth=1, transform=ax.transAxes, zorder=2))
    ax.text(0.5, y+0.04, name, transform=ax.transAxes, ha='center', fontsize=12, fontweight='bold', zorder=3)
    ax.text(0.5, y-0.04, val, transform=ax.transAxes, ha='center', fontsize=10, color='#555', zorder=3)
ax.set_title('구조 분석', fontweight='bold', fontsize=12)

# 2. Arm별 성과 비교
ax = fig.add_subplot(gs[0, 1:])
arms = ['Arm 1\n소분자 설계', 'Arm 2\n펩타이드 변이', 'Arm 3\nDe Novo 설계']
metrics = [0.94, 0.0, 81.4 / 100]  # normalized
metric_labels = ['QED = 0.94', 'FlexPepDock\n(미수행)', f'pLDDT = {max(d["plddt"] for d in designs):.1f}']
candidates = [40, 13, 16]
arm_colors = ['#42a5f5', '#66bb6a', '#ef5350']

x_pos = np.arange(len(arms))
bars = ax.bar(x_pos, candidates, color=arm_colors, edgecolor='white', linewidth=2, width=0.5)
for b, c, ml in zip(bars, candidates, metric_labels):
    ax.text(b.get_x()+b.get_width()/2, c+1, f'{c}개', ha='center', fontsize=13, fontweight='bold')
    ax.text(b.get_x()+b.get_width()/2, c/2, ml, ha='center', va='center', fontsize=9, color='white', fontweight='bold')
ax.set_xticks(x_pos); ax.set_xticklabels(arms, fontsize=11)
ax.set_ylabel('후보 분자 수', fontsize=11)
ax.set_title('3-Arm 파이프라인 비교', fontweight='bold', fontsize=13)

# 3. Top De Novo 서열
ax = fig.add_subplot(gs[1, 0:2])
ax.axis('off')
top5 = sorted_designs[:5]
for i, d in enumerate(top5):
    y = 0.88 - i * 0.18
    plddt = d['plddt']
    bar_width = plddt / 100 * 0.4
    color = '#2e7d32' if plddt >= 70 else '#ff9800'
    ax.add_patch(plt.Rectangle((0.55, y-0.03), bar_width, 0.07,
                                facecolor=color, alpha=0.6, transform=ax.transAxes))
    ax.text(0.02, y, f'{i+1}.', transform=ax.transAxes, fontsize=10, fontweight='bold', va='center')
    ax.text(0.06, y, d['binder_sequence'], transform=ax.transAxes,
            fontsize=10, fontfamily='monospace', va='center')
    ax.text(0.55 + bar_width + 0.02, y, f'pLDDT={plddt:.1f}', transform=ax.transAxes,
            fontsize=9, va='center', fontweight='bold', color=color)
ax.set_title('Top 5 De Novo 펩타이드 바인더', fontweight='bold', fontsize=12)

# 4. Next Steps
ax = fig.add_subplot(gs[1, 2])
ax.axis('off')
next_steps = [
    'AlphaFold3에서\n복합체 재예측',
    'PyRosetta\nFlexPepDock 실행',
    'MD Simulation으로\n안정성 평가',
    'In vitro\n바인딩 실험',
]
for i, step in enumerate(next_steps):
    y = 0.85 - i * 0.22
    color = '#1565c0' if i < 2 else '#9e9e9e'
    ax.text(0.15, y, f'{i+1}', transform=ax.transAxes, fontsize=14, fontweight='bold',
            ha='center', va='center', color='white',
            bbox=dict(boxstyle='circle', facecolor=color, edgecolor='white', linewidth=2))
    ax.text(0.3, y, step, transform=ax.transAxes, fontsize=10, va='center')
ax.set_title('Next Steps', fontweight='bold', fontsize=12)

fig.suptitle('SSTR2 Virtual Screening Pipeline — Final Dashboard',
             fontweight='bold', fontsize=16, y=1.02)
plt.show()

In [ ]:
display(HTML('''
<div style="background:linear-gradient(135deg,#1565c0,#42a5f5);color:white;padding:30px;border-radius:12px;margin:20px 0">
    <h2 style="margin-top:0">Key Takeaways</h2>
    <ol style="font-size:15px;line-height:1.8">
        <li><strong>AlphaFold3</strong>로 SSTR2-Somatostatin 복합체 구조를 높은 신뢰도(ipTM=0.71)로 예측</li>
        <li><strong>FoldMason</strong>으로 모델 간 구조 일관성 검증 (lDDT=0.664)</li>
        <li><strong>Binding Pocket</strong>: 35개 잔기, Trp8과 Lys9이 핵심 상호작용 잔기</li>
        <li><strong>Arm 1</strong>: MolMIM으로 QED 0.94 달성하는 drug-like 소분자 후보 40개 생성</li>
        <li><strong>Arm 2</strong>: 13개 변이체 설계, Alanine scanning으로 약효단 중요도 매핑</li>
        <li><strong>Arm 3</strong>: 16개 de novo 펩타이드 중 8개가 pLDDT > 70으로 고신뢰도 폴딩 확인</li>
    </ol>
    <p style="font-size:13px;opacity:0.8;margin-bottom:0">
        All NVIDIA NIM APIs (MolMIM, DiffDock, RFdiffusion, ProteinMPNN, ESMFold) — GPU 불필요, 클라우드 API 호출만으로 실행
    </p>
</div>
'''))